In [1]:
import sys
import scanpy as sc 
import numpy 

sys.path.append('/home/jupyter/data/PlastiNet_v2')

# plotting functions 
from plastinet.data.data_loader import create_data_objects

from plastinet.visualization.plots import plot_continous_obs, plot_tissue, plot_composition_dot_plot, plot_expression

/home/jupyter/mambaforge/envs/plastiNet/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jupyter/mambaforge/envs/plastiNet/lib/python3.8/site-packages/torch_geometric/typing.py:110: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: 
object has no attribute sparse_csc_tensor:
  File "/home/jupyter/mambaforge/envs/plastiNet/lib/python3.8/site-packages/torch_sparse/tensor.py", line 585
            value = torch.ones(self.nnz(), dtype=dtype, device=self.device())
    
        return torch.sparse_csc_tensor(colptr, row, value, self.sizes())
               ~~~~~~~~~~~~~~~~~~~~~~~ <--- HERE

  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [2]:
adata = sc.read_h5ad("../../C_back_end/dummy_data/multi_radial_adata.h5ad")
adata

/home/jupyter/mambaforge/envs/plastiNet/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 2200 × 3
    obs: 'R', 'G', 'B'
    obsm: 'spatial'

In [3]:
adata.obs['X'] = adata.obsm['spatial'][:, 0]  # X coordinate
adata.obs['Y'] = adata.obsm['spatial'][:, 1]  # Y coordinate
adata

AnnData object with n_obs × n_vars = 2200 × 3
    obs: 'R', 'G', 'B', 'X', 'Y'
    obsm: 'spatial'

In [4]:
adata.obs["sample_id"] = [0 for x in adata.obs.index]

In [16]:
import importlib
import torch
from torch_geometric.data import DataLoader

# Import the required modules and use `importlib.reload` to refresh them
from plastinet.models import plastinet_model
from plastinet.data import data_loader, graph_utils

# Reload the modules to refresh them without restarting the kernel
importlib.reload(plastinet_model)
importlib.reload(data_loader)
importlib.reload(graph_utils)

# Now re-import the specific classes and functions from the reloaded modules
from plastinet.models.plastinet_model import PlastiNet
from plastinet.data.data_loader import create_data_objects
from plastinet.data.graph_utils import graph_alpha, sparse_mx_to_torch_edge_list, graph_to_sparse_matrix

def run_plastinet_on_adata(adata, sample_key='sample_id', radius=50, z_dim=50, epochs=30, spatial_reg=0.2):
    """
    Wrapper function to run PlastiNet on an AnnData object.

    Args:
        adata (AnnData): AnnData object containing the spatial and gene expression data.
        sample_key (str): The key in adata.obs used to distinguish samples.
        radius (float): The neighborhood radius for spatial graph construction.
        z_dim (int): Dimensionality of the latent space for PlastiNet.
        epochs (int): Number of training epochs.
        spatial_reg (float): Spatial regularization weight.
        
    Returns:
        PlastiNet object with trained embeddings and attention weights.
    """
    # Create data objects for PyTorch Geometric DataLoader
    data_list = create_data_objects(
        adata=adata,
        sample_key=sample_key,
        radius = radius,
        graph_alpha=graph_alpha,
        sparse_mx_to_torch_edge_list=sparse_mx_to_torch_edge_list,
        graph_to_sparse_matrix=graph_to_sparse_matrix
    )

    # Initialize DataLoader
    dataloader = DataLoader(data_list, batch_size=1, shuffle=True)

    # Initialize and train PlastiNet
    plastinet = PlastiNet(adata=adata, sample_key=sample_key, radius=radius, z_dim=z_dim, epochs=epochs, spatial_reg=spatial_reg)
    plastinet.train(dataloader)

    return plastinet


IndentationError: expected an indented block (plastinet_model.py, line 119)

In [17]:
plastinet_model = run_plastinet_on_adata(
    adata=adata, 
    sample_key='sample_id',  # Modify based on your actual sample key
    radius=10, 
    z_dim=2, 
    epochs=1, 
    spatial_reg=0.2
)

# Access the resulting embeddings and attention weights
embedding_adata = plastinet_model.get_embedding_adata()

Epoch 0/1, Loss: 13.456686973571777


AttributeError: 'PlastiNet' object has no attribute 'get_embedding_adata'